In [8]:
import csv
from selenium import webdriver
from bs4 import BeautifulSoup


driver = webdriver.Chrome()


url = "https://misbar.com/"
driver.get(url)

driver.implicitly_wait(10)

html_content = driver.page_source

driver.quit()

soup = BeautifulSoup(html_content, "html.parser")

article_data = []

for link in soup.find_all('a', href=True):
    if '/editorial/' in link['href'] or '/factcheck/' in link['href']:
        article_url = "https://misbar.com/" + link['href']
        
        
        article_title = link.text.strip()
        
        article_data.append([article_title, article_url])

with open('articles.csv', 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['Topic', 'URL', 'type'])  
    csv_writer.writerows(article_data)

print("Les titres et les URLs des articles ont été enregistrés dans le fichier articles.csv.")

Les titres et les URLs des articles ont été enregistrés dans le fichier articles.csv.


In [9]:
import csv
import requests
from bs4 import BeautifulSoup


urls = [
    "http://norumors.net/",
    #"https://verify-sy.com/",
    "https://fatabyyano.net/"
]

def extract_articles(url):
    articles_data = []
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        article_titles = soup.find_all('h2')
        for title in article_titles:
            article_title = title.text.strip()  
            article_url = title.find_previous('a', href=True)
            if article_url:
                article_url = article_url['href']
            else:
                article_url = "Non disponible"
            articles_data.append([article_title, article_url])  
    
    with open('articles.csv', 'a', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        if csvfile.tell() == 0:  
        csv_writer.writerows(articles_data)

    print("Les titres et les URLs des articles ont été ajoutés dans le fichier articles.csv.")

for url in urls:
    print(f"Extraction des articles à partir de {url}\n")
    extract_articles(url)

Extraction des articles à partir de http://norumors.net/

Les titres et les URLs des articles ont été ajoutés dans le fichier articles.csv.
Extraction des articles à partir de https://fatabyyano.net/

Les titres et les URLs des articles ont été ajoutés dans le fichier articles.csv.


In [7]:
import csv
import requests
from bs4 import BeautifulSoup


base_url1 = "https://verify-sy.com/list/112/%D9%85%D8%A4%D9%83%D8%AF?page={}"
base_url2 = "https://verify-sy.com/list/16/%D8%A7%D8%AD%D8%AA%D9%8A%D8%A7%D9%84?page={}"


total_pages = 3


all_articles_data = []


def extract_articles(url, article_type):
    articles_data = []
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        h3_tags = soup.find_all('h3', class_='list-title-ca')
        for h3_tag in h3_tags:
            link = h3_tag.find('a')
            if link:
                article_url = link['href']
                article_title = link.text.strip()
                articles_data.append([article_title, article_url, article_type])
    return articles_data


for page_number in range(1, total_pages + 1):
    url = base_url1.format(page_number)
    all_articles_data.extend(extract_articles(url, 'reel'))


for page_number in range(1, total_pages + 1):
    url = base_url2.format(page_number)
    all_articles_data.extend(extract_articles(url, 'fake'))


with open('articles.csv', 'a', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    
    csv_writer.writerows(all_articles_data)

print("Toutes les données ont été enregistrées dans le fichier articles.csv.")

Toutes les données ont été enregistrées dans le fichier articles.csv.


In [25]:
import csv


categories_arabic = {
    'Politics': ['الجيش','وزارة','حروب','لاجئة','قتلت','الدولي','الأمة','سياسة', 'انتخابات', 'حكومة', 'برلمان','الدولة', 'ساسة', 'نظام سياسي', 'ثورة', 'ديمقراطية', 'حرب', 'دبلوماسية'],
    'Religion': ['السلامة','الله','دين', 'إيمان', 'روحانية', 'معبد', 'الكنيسة', 'ديانة', 'مسيحي', 'إسلامي', 'صلاة', 'قرآن', 'رمضان', 'صوم'],
    'Health': ['دماء','السرطان','جنس','طبي','صحة', 'مرض', 'طب', 'تغذية', 'وباء', 'صحي', 'علاج', 'مستشفى', 'لقاح', 'نظام غذائي', ' صحي', ' طبية'],
    'Technology': ['تكنولوجيا', 'إنترنت', 'تطبيق', 'رقمي', 'ذكاء اصطناعي', 'تقنية', 'تطوير', 'برمجة', 'شبكات اجتماعية', 'تشفير', 'هواتف ذكية', 'تطبيقات محمولة'],
    'Sports': ['رياضة', 'كرة القدم', 'تنس', 'سباحة', 'ألعاب أولمبية', 'مباراة', 'فريق', 'تدريب', 'ماراثون', 'جمباز', 'محترفون', ' رياضية'],
    'Business': ['أعمال', 'تجارة', 'اقتصاد', 'تطوير الأعمال', ' المال', 'شركة', 'استثمار', 'ريادة الأعمال', 'تسويق', 'ربحية', 'إدارة أعمال', ' توريد'],
    
}


def categorize_article(title):
    category = None
    
    for category_arabic, keywords in categories_arabic.items():
        for keyword in keywords:
            if keyword in title:
                category = category_arabic
                break
    
    if category is None:
        category = 'Other'  
                
    return category


categorized_articles = []
with open('articles.csv', 'r', newline='', encoding='utf-8') as csvfile:
    csv_reader = csv.reader(csvfile)
    next(csv_reader)  # Ignorer l'en-tête
    for row in csv_reader:
        title = row[0] if len(row) >= 1 else ''  
        url = row[1] if len(row) >= 2 else ''  
        article_type = row[2] if len(row) >= 3 else ''  
        category = categorize_article(title)
        categorized_articles.append([title, url, article_type, category])

        if article_type == '':
            categorized_articles.append([title, url, article_type, category])


with open('Data_Collection.csv', 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['Title', 'URL', 'Type', 'Category'])  
    csv_writer.writerows(categorized_articles)

print("Les articles ont été catégorisés et enregistrés ")

Les articles ont été catégorisés et enregistrés 


In [26]:
from collections import Counter

# Initialize a counter to store the count of each category
category_counter = Counter()

# Iterate through the categorized articles and count occurrences of each category
for article in categorized_articles:
    category = article[3]  # The category is stored at index 3
    category_counter[category] += 1

# Print the count of articles in each category
for category, count in category_counter.items():
    print(f"{category}: {count}")

Religion: 13
Politics: 25
Health: 19
Other: 120
Sports: 4
Business: 2
